In [ ]:

import os
import numpy as np
from pydub import AudioSegment
import librosa
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Flatten
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


In [ ]:

# Function to convert MP3 to WAV
def convert_mp3_to_wav(mp3_file, wav_file):
    audio = AudioSegment.from_mp3(mp3_file)
    audio.export(wav_file, format="wav")


In [ ]:

# Function to load raw audio waveform
def load_audio_waveform(file_name, max_length=22050):
    audio, sample_rate = librosa.load(file_name, sr=22050)
    if len(audio) > max_length:
        audio = audio[:max_length]
    elif len(audio) < max_length:
        audio = np.pad(audio, (0, max_length - len(audio)))
    return audio


In [ ]:

# Load dataset
def load_data(mp3_files, labels, max_length=22050):
    features = []
    for file in mp3_files:
        wav_file = file.replace('.mp3', '.wav')
        convert_mp3_to_wav(file, wav_file)
        features.append(load_audio_waveform(wav_file, max_length))
    return np.array(features), np.array(labels)

# Example usage
mp3_files = ['file1.mp3', 'file2.mp3']  # Replace with your mp3 files
labels = ['AI', 'Human']  # Corresponding labels


In [ ]:

# Preprocess data
X, y = load_data(mp3_files, labels)
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y = tf.keras.utils.to_categorical(y, num_classes=2)

# Reshape data for CNN
X = X.reshape(X.shape[0], X.shape[1], 1)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:

# Build CNN model
model = Sequential()
model.add(Conv1D(16, kernel_size=3, activation='relu', input_shape=(X.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:

# Train model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))


In [ ]:

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy*100:.2f}%")
